In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset='/content/drive/MyDrive/ML/bot_detection_data.csv'
import pandas as pd
df=pd.read_csv(dataset)

In [40]:
df

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,0,1,Adkinston,2020-05-11 15:29:50,No Hashtag
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,1,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,1,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,1,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,0,1,Camachoville,2020-04-13 21:24:21,foreign mention
...,...,...,...,...,...,...,...,...,...,...,...
49995,491196,uberg,Want but put card direction know miss former h...,64,0,9911,1,1,Lake Kimberlyburgh,2023-04-20 11:06:26,teach quality ten education any
49996,739297,jessicamunoz,Provide whole maybe agree church respond most ...,18,5,9900,0,1,Greenbury,2022-10-18 03:57:35,add walk among believe
49997,674475,lynncunningham,Bring different everyone international capital...,43,3,6313,1,1,Deborahfort,2020-07-08 03:54:08,onto admit artist first
49998,167081,richardthompson,Than about single generation itself seek sell ...,45,1,6343,0,0,Stephenside,2022-03-22 12:13:44,star


In [ ]:
df.info()

In [ ]:
print(df.isna().sum())

In [ ]:
print(df.isnull().sum())

In [ ]:
#adding our own data in missing hastag rows

missing_values_mask=df['Hashtags'].isnull()
#print(missing_values_mask)
df.loc[missing_values_mask,'Hashtags']='No Hashtag'#add No Hashtag at missing cell
print(df['Hashtags'])

In [ ]:
#changing datatype of column 'Created At'

df['Created At']=pd.to_datetime(df['Created At'], format='%Y-%m-%d %H:%M')
print(df['Created At'].dtypes)

datetime64[ns]


In [ ]:
#summary statistics
df.describe()

In [ ]:
df

In [ ]:
# Assuming you have a DataFrame named 'data' with multiple columns

# List of columns to drop
columns_to_drop = ['User ID', 'Retweet Count', 'Mention Count','Follower Count','Verified','Bot Label','Hashtags']

# Drop the specified columns from the DataFrame
data_dropped = df.drop(columns=columns_to_drop)

# Get value counts for each remaining column
column_counts = {}
for column in data_dropped.columns:
    column_counts[column] = data_dropped[column].value_counts()

# Print the value counts for each column
for column, counts in column_counts.items():
    print(f"Value counts for {column}:")
    print(counts)
    print()



In [ ]:
#identify inconsistent formatting for textual(categorical too) columns

print(df['Hashtags'].value_counts())


In [ ]:
import matplotlib.pyplot as plt

# Filter data based on bot label
bot_data = df[df['Bot Label'] == 1]
non_bot_data = df[df['Bot Label'] == 0]





In [ ]:
bot_followers_stats = bot_data['Follower Count'].describe()
non_bot_followers_stats = non_bot_data['Follower Count'].describe()

bot_retweet_stats = bot_data['Retweet Count'].describe()
non_bot_retweet_stats = non_bot_data['Retweet Count'].describe()

bot_mention_stats = bot_data['Mention Count'].describe()
non_bot_mention_stats = non_bot_data['Mention Count'].describe()


print("Follower Count Statistics:")
print("Bot Accounts:")
print(bot_followers_stats)
print("\nNon-Bot Accounts:")
print(non_bot_followers_stats)

print("\nRetweet Count Statistics:")
print("Bot Accounts:")
print(bot_retweet_stats)
print("\nNon-Bot Accounts:")
print(non_bot_retweet_stats)

print("\nMention Count Statistics:")
print("Bot Accounts:")
print(bot_mention_stats)
print("\nNon-Bot Accounts:")
print(non_bot_mention_stats)

In [ ]:
#label encoding of boolean data
from sklearn.preprocessing import LabelEncoder

label_encoder=LabelEncoder()
df['Verified']=label_encoder.fit_transform(df['Verified'])
df

In [ ]:
df

In [ ]:
df.info()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
# Combine all text columns into a single Series
text_data = df['Tweet'] + ' ' + df['Username'] + ' ' + df['Hashtags']+' '+df['Location']

# Text vectorization using TF-IDF (sparse representation)
vectorizer = TfidfVectorizer()
text_sparse = vectorizer.fit_transform(text_data)






In [ ]:
# Combine text features with additional features

additional_features = df[['Retweet Count', 'Verified','Mention Count','Follower Count','Created At']]

In [ ]:
additional_features['Created At'] = additional_features['Created At'].astype(int)  # Convert to Unix timestamp
additional_features = additional_features.astype('float64')  # Convert to float64
text_sparse = text_sparse.astype('float64')

In [ ]:


combined_sparse = hstack((text_sparse, additional_features))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_sparse, df['Bot Label'], test_size=0.2, random_state=42)

# Create an instance of the Random Forest classifier
rf_classifier = RandomForestClassifier()

# Train the Random Forest classifier
rf_classifier.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
# Make predictions on the testing data
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Accuracy: 0.5027
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.53      0.52      4968
           1       0.51      0.47      0.49      5032

    accuracy                           0.50     10000
   macro avg       0.50      0.50      0.50     10000
weighted avg       0.50      0.50      0.50     10000



In [53]:
# Example: Predict labels for new data
from datetime import datetime



# New data (example)

new_data = pd.DataFrame({

    'Tweet': ['Just cover eight opportunity strong policy which.'],

    'Username': ['pmason'],

    'Hashtags': ['neever quickly new Iw'],

    'Retweet Count': [54],

    'Verified': [1],

    'Location' : ['Martinezberg'],

    'Mention Count' : [5],

    'Follower Count' : [2242],

    'Created At' : ['14-08-2021  22:27:00']
})




# Concatenate text features

new_text_data = new_data['Tweet'] + ' ' + new_data['Username'] + ' ' + new_data['Hashtags']+' '+new_data['Location']




# Perform feature vectorization on new text features

new_text_sparse = vectorizer.transform(new_text_data)




# Combine new text features with additional features

new_additional_features = new_data[['Retweet Count', 'Verified','Mention Count','Follower Count','Created At']]



new_additional_features['Created At'] = pd.to_datetime(new_additional_features['Created At'])
new_additional_features['Created At'] = new_additional_features['Created At'].apply(lambda x: int(datetime.timestamp(x)))

# new_additional_features = new_additional_features.astype('float64')


# new_additional_features['Created At'] = new_additional_features['Created At'].astype(int)  # Convert to Unix timestamp
new_additional_features = new_additional_features.astype('float64')

new_combined_sparse = hstack((new_text_sparse, new_additional_features))




# Predict labels for the new data

new_predictions = rf_classifier.predict(new_combined_sparse)




# Print the predictions

for i, prediction in enumerate(new_predictions):

    if prediction == 1:

        print(f"Data point {i+1}: Bot")

    else:

        print(f"Data point {i+1}: Not Bot")

Data point 1: Bot


<ipython-input-53-2e724744b973>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_additional_features['Created At'] = pd.to_datetime(new_additional_features['Created At'])
<ipython-input-53-2e724744b973>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_additional_features['Created At'] = new_additional_features['Created At'].apply(lambda x: int(datetime.timestamp(x)))


**EXTRA(EXPERIMENTS)**

In [ ]:
# Example: Predict if a new tweet is a bot or not
new_tweet = ["Just cover eight opportunity strong policy which."]
new_tweet_sparse = vectorizer.transform(new_tweet)

prediction = rf_classifier.predict(new_tweet_sparse)

if prediction[0] == 1:
   print("The tweet is predicted to be from a bot.")
else:
   print("The tweet is predicted to be from a non-bot.")

The tweet is predicted to be from a bot.


In [ ]:
#EXTRA

In [ ]:
Y=df['Bot Label']
columns_drop=['User ID','Bot Label','Retweet Count','Mention Count','Follower Count','Verified','Created At']#Give only textual column as we will preprocess the text data
X=df.drop(columns_drop,axis=1)

In [ ]:
type(Y)

pandas.core.series.Series

In [ ]:
type(X)

pandas.core.frame.DataFrame

In [ ]:
#X=X.stack()

In [ ]:
type(X)

pandas.core.frame.DataFrame

In [ ]:
Y

0        1
1        0
2        0
3        1
4        1
        ..
49995    1
49996    1
49997    1
49998    0
49999    0
Name: Bot Label, Length: 50000, dtype: int64

In [ ]:
X

,Username,Tweet,Location,Hashtags
0,flong,Station activity person against natural majori...,Adkinston,No Hashtag
1,hinesstephanie,Authority research natural life material staff...,Sanderston,both live
2,roberttran,Manage whose quickly especially foot none to g...,Harrisonfurt,phone ahead
3,pmason,Just cover eight opportunity strong policy which.,Martinezberg,ever quickly new I
4,noah87,Animal sign six data good or.,Camachoville,foreign mention
...,...,...,...,...
49995,uberg,Want but put card direction know miss former h...,Lake Kimberlyburgh,teach quality ten education any
49996,jessicamunoz,Provide whole maybe agree church respond most ...,Greenbury,add walk among believe
49997,lynncunningham,Bring different everyone international capital...,Deborahfort,onto admit artist first
49998,richardthompson,Than about single generation itself seek sell ...,Stephenside,star


In [ ]:
#splitting
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [ ]:
type(X_train)

pandas.core.frame.DataFrame

In [ ]:
X_train=X_train.values.flatten()

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
X_test=X_test.values.flatten()

In [ ]:
X_train

array(['kristaperkins',
       'Unit read window my radio medical yes lawyer similar room.',
       'East Sherry', ...,
       'He add street foreign development among worker end key doctor land term.',
       'Kellyburgh', 'yourself'], dtype=object)

In [ ]:
X_train.shape

(150000,)

In [ ]:
X_test

array(['wellsvicki', 'Right collection site require nearly we available.',
       'South Jessicaport', ...,
       'Else good bring charge example whether room.', 'South Brian',
       'officer fish'], dtype=object)

In [ ]:
X_test.shape

(50000,)

In [ ]:
df.info()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector=CountVectorizer()
#fir the training data & transform & this return matrix -1D array as param
train_data=count_vector.fit_transform(X_train)
test_data=count_vector.transform(X_test)

In [ ]:
print(train_data)

  (0, 23514)	1
  (1, 41537)	1
  (1, 33739)	1
  (1, 43784)	1
  (1, 29755)	1
  (1, 33413)	1
  (1, 27592)	1
  (1, 44505)	1
  (1, 24430)	1
  (1, 37710)	1
  (1, 35421)	1
  (2, 11569)	1
  (2, 37567)	1
  (3, 5603)	1
  (3, 6348)	1
  (3, 36934)	1
  (3, 38429)	1
  (4, 41952)	1
  (5, 35421)	1
  (5, 16822)	1
  (5, 32088)	1
  (5, 30967)	1
  (5, 12826)	1
  (5, 15778)	1
  (5, 25512)	1
  :	:
  (149993, 32289)	1
  (149993, 7913)	1
  (149993, 25199)	1
  (149993, 16153)	1
  (149994, 11569)	1
  (149994, 1354)	1
  (149995, 41783)	1
  (149995, 10880)	1
  (149995, 37800)	1
  (149995, 30970)	1
  (149996, 12163)	1
  (149997, 10640)	1
  (149997, 16152)	1
  (149997, 39786)	1
  (149997, 10982)	1
  (149997, 38984)	1
  (149997, 43983)	1
  (149997, 307)	1
  (149997, 1344)	1
  (149997, 24002)	1
  (149997, 12247)	1
  (149997, 22984)	1
  (149997, 13385)	1
  (149998, 22546)	1
  (149999, 44712)	1


In [ ]:
print(test_data)

  (1, 2837)	1
  (1, 8167)	1
  (1, 30125)	1
  (1, 34048)	1
  (1, 34494)	1
  (1, 37829)	1
  (1, 42908)	1
  (2, 19637)	1
  (2, 38172)	1
  (3, 16013)	1
  (3, 30672)	1
  (4, 35167)	1
  (5, 8972)	1
  (5, 11554)	1
  (5, 12832)	1
  (5, 13352)	1
  (5, 26027)	1
  (5, 30315)	1
  (5, 32786)	1
  (5, 34054)	1
  (5, 37884)	1
  (5, 38440)	1
  (5, 41121)	1
  (5, 43329)	1
  (7, 5969)	1
  :	:
  (49993, 23048)	1
  (49993, 26991)	1
  (49993, 31145)	1
  (49993, 31419)	1
  (49993, 37661)	1
  (49993, 39729)	1
  (49993, 40828)	1
  (49994, 7474)	1
  (49994, 38172)	1
  (49995, 6680)	1
  (49995, 10637)	1
  (49995, 40828)	1
  (49995, 41105)	1
  (49996, 2736)	1
  (49997, 5010)	1
  (49997, 6850)	1
  (49997, 12113)	1
  (49997, 12821)	1
  (49997, 14864)	1
  (49997, 35421)	1
  (49997, 43203)	1
  (49998, 4775)	1
  (49998, 38172)	1
  (49999, 13129)	1
  (49999, 30971)	1


In [ ]:
train_data.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
test_data.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
count_vector.get_feature_names_out(test_data)

array(['aabbott', 'aaguilar', 'aaguirre', ..., 'zwright', 'zyoung',
       'zzamora'], dtype=object)

In [ ]:
count_vector.get_feature_names_out(train_data)

array(['aabbott', 'aaguilar', 'aaguirre', ..., 'zwright', 'zyoung',
       'zzamora'], dtype=object)

In [ ]:
type(train_data)

scipy.sparse._csr.csr_matrix

In [ ]:
train_data.size


556339

In [ ]:
train_data.shape

(150000, 45191)

In [ ]:
type(Y_train)

pandas.core.series.Series

In [ ]:
Y_train.size

37500

In [ ]:
Y_train.shape

(37500,)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create an instance of the DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier()

# Fit the model on the training data
dt_classifier.fit(train_data, Y_train)

ValueError: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on the training data
rf_classifier.fit(train_data, Y_train)


ValueError: ignored

In [ ]:
Username=df['Username']
Tweet=df['Tweet']
Location=df['Location']
Hashtags=df['Hashtags']

In [ ]:
Username

0                  flong
1         hinesstephanie
2             roberttran
3                 pmason
4                 noah87
              ...       
49995              uberg
49996       jessicamunoz
49997     lynncunningham
49998    richardthompson
49999           daniel29
Name: Username, Length: 50000, dtype: object

In [ ]:
Username_list= Username.tolist()
Tweet_list=Tweet.tolist()
Location_list=Location.tolist()
Hashtags_list=Hashtags.tolist()

In [ ]:
Username_list[0]

'flong'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
print(cv)

CountVectorizer()


In [ ]:
cv.fit(Username_list)

CountVectorizer()

In [ ]:
cv.get_feature_names_out()

array(['aabbott', 'aaguilar', 'aaguirre', ..., 'zwyatt', 'zyoung',
       'zzamora'], dtype=object)

In [ ]:
cv.transform(Username_list).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
matrix=pd.DataFrame(cv.transform(Username_list).toarray(),columns=cv.get_feature_names_out())
matrix

,ability,able,about,above,accept,according,account,across,act,action,...,wrong,yard,yeah,year,yes,yet,you,young,your,yourself
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
cv.fit(Tweet_list)

CountVectorizer()

In [ ]:
cv.get_feature_names_out()

array(['ability', 'able', 'about', 'above', 'accept', 'according',
       'account', 'across', 'act', 'action', 'activity', 'actually',
       'add', 'address', 'administration', 'admit', 'adult', 'affect',
       'after', 'again', 'against', 'age', 'agency', 'agent', 'ago',
       'agree', 'agreement', 'ahead', 'air', 'all', 'allow', 'almost',
       'alone', 'along', 'already', 'also', 'although', 'always',
       'american', 'among', 'amount', 'analysis', 'and', 'animal',
       'another', 'answer', 'any', 'anyone', 'anything', 'appear',
       'apply', 'approach', 'area', 'argue', 'arm', 'around', 'arrive',
       'art', 'article', 'artist', 'as', 'ask', 'assume', 'at', 'attack',
       'attention', 'attorney', 'audience', 'author', 'authority',
       'available', 'avoid', 'away', 'baby', 'back', 'bad', 'bag', 'ball',
       'bank', 'bar', 'base', 'be', 'beat', 'beautiful', 'because',
       'become', 'bed', 'before', 'begin', 'behavior', 'behind',
       'believe', 'benefit', 'be

In [ ]:
cv.transform(Tweet_list).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df['Username']
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
tfidf_vectors = vectorizer.transform(corpus)
tfidf_vectors = tfidf_vectors.toarray()

# Add more columns as needed, separated by space


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Perform TF-IDF vectorization on each column separately
for column in df.columns:
    # Retrieve the text data from the column
    text_data = df[column].astype(str)

    # Fit the vectorizer to the text data
    tfidf_vectors=vectorizer.fit_transform(text_data)

    # Convert the sparse matrix to a dense array (if desired)
    tfidf_vectors = tfidf_vectors.toarray()

    # Access the feature names (terms) and their corresponding indices
    feature_names = vectorizer.get_feature_names_out()



ValueError: ignored

In [ ]:
vectorizer.get_feature_names_out()

array(['aabbott', 'aaguilar', 'aaguirre', ..., 'zwyatt', 'zyoung',
       'zzamora'], dtype=object)

In [ ]:
tfidf_vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
matrix=pd.DataFrame(vectorizer.transform(corpus).toarray(),columns=vectorizer.get_feature_names_out())
matrix

,aabbott,aaguilar,aaguirre,aalvarez,aanderson,aanthony,aarmstrong,aaron00,aaron07,aaron08,...,zwells,zwhite,zwilliams,zwilliamson,zwilson,zwood,zwright,zwyatt,zyoung,zzamora
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
